This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/acrobot.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    Propeller,
    PropellerInfo,
    RigidTransform,
    RobotDiagramBuilder,
    SceneGraph,
    Simulator,
    StartMeshcat,
    namedview,
)
from pydrake.examples import QuadrotorGeometry, QuadrotorPlant, StabilizingLQRController

from underactuated import running_as_notebook
from underactuated.scenarios import AddFloatingRpyJoint

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# LQR for the 3D Quadrotor

Art assets courtesy of our friends at Skydio.

In [ ]:
def quadrotor_example():
    builder = DiagramBuilder()

    plant = builder.AddSystem(QuadrotorPlant())

    controller = builder.AddSystem(StabilizingLQRController(plant, [0, 0, 1]))
    builder.Connect(controller.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), controller.get_input_port(0))

    # Set up visualization in MeshCat
    scene_graph = builder.AddSystem(SceneGraph())
    QuadrotorGeometry.AddToBuilder(builder, plant.get_output_port(0), scene_graph)
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty("/Background", "visible", False)
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    # end setup for visualization

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    context = simulator.get_mutable_context()

    # Simulate
    for i in range(5):
        context.SetTime(0.0)
        context.SetContinuousState(
            0.5
            * np.random.randn(
                12,
            )
        )
        simulator.Initialize()
        simulator.AdvanceTo(4.0 if running_as_notebook else 0.1)


quadrotor_example()

## The 3D Quadrotor again, now using MultibodyPlant

The example above used [`QuadrotorPlant`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1examples_1_1quadrotor_1_1_quadrotor_plant.html), in which I've typed in the equations of motion by hand.  This is a great way to work with the system, but is hard to generalize.

If you want to add additional forces to the model -- for instance if you want the quadrotor to have contact dynamics with the world (to land on the ground, or collide with a tree) or if you want to add e.g. wind -- then we need a more general solution.  We can also implement the quadrotor dynamics using `MultibodyPlant`.  We just need to manually wire the [`Propeller`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_propeller.html) forces into the `Diagram` (because `Propeller` is not a concept supported in URDF nor SDF just yet).

In [ ]:
def MakeMultibodyQuadrotor():
    builder = DiagramBuilder()
    # The MultibodyPlant handles f=ma, but doesn't know about propellers.
    plant = builder.AddSystem(MultibodyPlant(0.0))
    parser = Parser(plant)
    (model_instance,) = parser.AddModelsFromUrl(
        "package://drake_models/skydio_2/quadrotor.urdf"
    )
    # By default the multibody has a quaternion floating base.  To match
    # QuadrotorPlant, we can manually add a FloatingRollPitchYaw joint. We set
    # `use_ball_rpy` to false because the BallRpyJoint uses angular velocities
    # instead of ṙ, ṗ, ẏ.
    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName("base_link"),
        model_instance,
        use_ball_rpy=False,
    )
    plant.Finalize()

    # Now we can add in propellers as an external force on the MultibodyPlant.
    body_index = plant.GetBodyByName("base_link").index()
    # Default parameters from quadrotor_plant.cc:
    L = 0.15  # Length of the arms (m).
    kF = 1.0  # Force input constant.
    kM = 0.0245  # Moment input constant.

    # Note: Rotors 0 and 2 rotate one way and rotors 1 and 3 rotate the other.
    prop_info = [
        PropellerInfo(body_index, RigidTransform([L, 0, 0]), kF, kM),
        PropellerInfo(body_index, RigidTransform([0, L, 0]), kF, -kM),
        PropellerInfo(body_index, RigidTransform([-L, 0, 0]), kF, kM),
        PropellerInfo(body_index, RigidTransform([0, -L, 0]), kF, -kM),
    ]
    propellers = builder.AddSystem(Propeller(prop_info))
    builder.Connect(
        propellers.get_output_port(),
        plant.get_applied_spatial_force_input_port(),
    )
    builder.Connect(
        plant.get_body_poses_output_port(),
        propellers.get_body_poses_input_port(),
    )
    builder.ExportInput(propellers.get_command_input_port(), "u")

    return builder.Build(), plant


# This test demonstrates that the MultibodyPlant version has identical dynamics
# to the QuadrotorPlant version (except that the state variables are permuted).
# TODO(russt): Move this to Drake as a unit test.
def MultibodyQuadrotorExample():
    mbp_plant, mbp = MakeMultibodyQuadrotor()

    hand_derived_plant = QuadrotorPlant()

    # Compare the dynamics at a handful of states.
    mbp_plant_context = mbp_plant.CreateDefaultContext()
    mbp_context = mbp.GetMyContextFromRoot(mbp_plant_context)
    hand_derived_context = hand_derived_plant.CreateDefaultContext()

    # Permute mbp <=> hand_derived states.
    P = np.array(
        [
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 1],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 1, 0, 0],
        ]
    )
    PP = np.block([[P, np.zeros((6, 6))], [np.zeros((6, 6)), P]])

    rng = np.random.default_rng(seed=1037)
    for i in range(5):
        u = rng.random((4,))
        mbp_x = rng.random((12,))
        mbp_plant.get_input_port().FixValue(mbp_plant_context, u)
        mbp_context.SetContinuousState(mbp_x)
        mbp_xdot = mbp_plant.EvalTimeDerivatives(mbp_plant_context).CopyToVector()

        hand_derived_x = PP @ mbp_x
        hand_derived_plant.get_input_port().FixValue(hand_derived_context, u)
        hand_derived_context.SetContinuousState(hand_derived_x)
        hand_derived_xdot = (
            PP
            @ hand_derived_plant.EvalTimeDerivatives(
                hand_derived_context
            ).CopyToVector()
        )

        assert np.allclose(
            mbp_xdot, hand_derived_xdot
        ), f"\nmbp\t\t = {mbp_xdot}\nhand_derived\t = {hand_derived_xdot}"


MultibodyQuadrotorExample()

Now we can use this in LQR and all of our other tools...

In [ ]:
def MultibodyQuadrotorLQR():
    quadrotor, mbp = MakeMultibodyQuadrotor()
    # We'll use a namedview to make it easier to work with the state.
    StateView = namedview("state", mbp.GetStateNames(False))

    # Create the LQR controller
    context = quadrotor.CreateDefaultContext()
    nominal_state = StateView.Zero()
    nominal_state.z_x = 1.0  # height is 1.0m
    context.SetContinuousState(nominal_state[:])
    mass = mbp.CalcTotalMass(mbp.GetMyContextFromRoot(context))
    gravity = mbp.gravity_field().gravity_vector()[2]
    nominal_input = [-mass * gravity / 4] * 4
    quadrotor.get_input_port().FixValue(context, nominal_input)
    Q = np.diag(np.concatenate(([10] * 6, [1] * 6)))
    R = np.eye(4)
    LinearQuadraticRegulator(quadrotor, context, Q, R)


MultibodyQuadrotorLQR()

# Multiple quadrotors

We've had a number of great projects over the years using multiple quadrotors. Here's a simple example of running LQR on multiple quadrotors simultaneously.  One could also do this by having two separate LQR controllers, both feeding into the propellers, but the wiring on this version is clean and simple.

In [ ]:
def MakeMultipleQuadrotors():
    # The RobotDiagram is a convenient way to work with a MultibodyPlant +
    # SceneGraph.
    builder = RobotDiagramBuilder(time_step=0.0)
    plant = builder.plant()

    directives = """
    directives:
    - add_model:
        name: quad1
        file: package://drake_models/skydio_2/quadrotor.urdf
    - add_model:
        name: quad2
        file: package://drake_models/skydio_2/quadrotor.urdf
    """

    builder.parser().SetAutoRenaming(True)
    builder.parser().AddModelsFromString(directives, ".dmd.yaml")
    # Floating joints are easier for LQR.
    quad1 = plant.GetModelInstanceByName("quad1")
    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName("base_link", quad1),
        quad1,
        use_ball_rpy=False,
    )
    quad2 = plant.GetModelInstanceByName("quad2")
    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName("base_link", quad2),
        quad2,
        use_ball_rpy=False,
    )

    plant.Finalize()

    # Now we can add in propellers as an external force on the MultibodyPlant.
    quad1_body = plant.GetBodyByName("base_link", quad1).index()
    quad2_body = plant.GetBodyByName("base_link", quad2).index()
    # Default parameters from quadrotor_plant.cc:
    L = 0.15  # Length of the arms (m).
    kF = 1.0  # Force input constant.
    kM = 0.0245  # Moment input constant.

    # Note: Rotors 0 and 2 rotate one way and rotors 1 and 3 rotate the other.
    prop_info = [
        PropellerInfo(quad1_body, RigidTransform([L, 0, 0]), kF, kM),
        PropellerInfo(quad1_body, RigidTransform([0, L, 0]), kF, -kM),
        PropellerInfo(quad1_body, RigidTransform([-L, 0, 0]), kF, kM),
        PropellerInfo(quad1_body, RigidTransform([0, -L, 0]), kF, -kM),
        PropellerInfo(quad2_body, RigidTransform([L, 0, 0]), kF, kM),
        PropellerInfo(quad2_body, RigidTransform([0, L, 0]), kF, -kM),
        PropellerInfo(quad2_body, RigidTransform([-L, 0, 0]), kF, kM),
        PropellerInfo(quad2_body, RigidTransform([0, -L, 0]), kF, -kM),
    ]
    propellers = builder.builder().AddSystem(Propeller(prop_info))
    builder.builder().Connect(
        propellers.get_output_port(),
        plant.get_applied_spatial_force_input_port(),
    )
    builder.builder().Connect(
        plant.get_body_poses_output_port(),
        propellers.get_body_poses_input_port(),
    )
    builder.builder().ExportInput(propellers.get_command_input_port(), "u")
    builder.builder().ExportOutput(plant.get_state_output_port(), "x")
    builder.builder().ExportOutput(
        builder.scene_graph().get_query_output_port(), "query"
    )

    return builder.Build()


def MultipleQuadrotorDemo():
    builder = DiagramBuilder()
    robot_diagram = builder.AddSystem(MakeMultipleQuadrotors())
    plant = robot_diagram.plant()

    # We'll use a namedview to make it easier to work with the state.
    StateView = namedview("state", plant.GetStateNames(add_model_instance_prefix=True))
    x0 = StateView.Zero()
    x0.quad1_x_x = -1
    x0.quad1_z_x = 1
    x0.quad2_x_x = 1
    x0.quad2_z_x = 1
    plant.SetDefaultPositions(x0[: plant.num_positions()])

    # Create the LQR controller
    robot_diagram_context = robot_diagram.CreateDefaultContext()
    mass = plant.CalcTotalMass(
        plant.GetMyContextFromRoot(robot_diagram_context),
        [plant.GetModelInstanceByName("quad1")],
    )
    gravity = plant.gravity_field().gravity_vector()[2]
    nominal_input = [-mass * gravity / 4] * 8
    robot_diagram.get_input_port().FixValue(robot_diagram_context, nominal_input)
    Q = np.diag(np.concatenate(([10] * 12, [1] * 12)))
    R = np.eye(8)
    print(robot_diagram_context)
    controller = builder.AddSystem(
        LinearQuadraticRegulator(robot_diagram, robot_diagram_context, Q, R)
    )
    builder.Connect(controller.get_output_port(), robot_diagram.get_input_port())
    builder.Connect(robot_diagram.GetOutputPort("x"), controller.get_input_port())

    MeshcatVisualizer.AddToBuilder(
        builder, robot_diagram.GetOutputPort("query"), meshcat
    )
    diagram = builder.Build()

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    context = simulator.get_mutable_context()

    # Simulate
    for i in range(5):
        context.SetTime(0.0)
        context.SetContinuousState(
            0.5
            * np.random.randn(
                24,
            )
        )
        simulator.Initialize()
        simulator.AdvanceTo(4.0 if running_as_notebook else 0.1)


MultipleQuadrotorDemo()